# convert_a1

Notebook to convert whoi_ysi_log and backup files into a1-formatted files

@author: David Clemens-Sewall

In [1]:
# Imports
import os

import numpy as np
import pandas as pd

In [5]:
# Paths
data_path = os.path.join('..', '..', 'data', 'whoi_ysi')

log_file = 'whoi_ysi_log_final.xlsx'

In [6]:
# Load log
df_log = pd.read_excel(os.path.join(data_path, log_file))

In [7]:
# Scratchwork to figure out parsing

In [8]:
# Add columns to log file
df_log['timestamp_end'] = pd.to_datetime(df_log.date + ' ' + df_log.end_time)
df_log['timestamp_ctd_start'] = pd.NaT
df_log['timestamp_ctd_end'] = pd.NaT
df_log['visit'] = ''

In [14]:
# Add all start and end timestamps to log dataframe
stations = ['1a', '2a', '3a', '1b', '2b', '3b', '1c', '2c', '3c', '2d', '3d']

for station in stations:
    ## Load backup dataframe
    filenames = os.listdir(os.path.join(data_path, 'station'+station))
    ls_df = []
    for filename in filenames:
        if filename[:3] == 'csv':
            ls_df.append(pd.read_csv(os.path.join(data_path, 'station'+station, filename)))
    df_bkp = pd.concat(ls_df)
    df_bkp['timestamp_utc'] = pd.to_datetime('20' + df_bkp['Date'] + ' ' + df_bkp['Time'])
    
    ## parse ctd start and end times
    timestamp_bks = df_bkp[df_bkp['timestamp_utc'].diff(-1) < pd.Timedelta(seconds=-30)]['timestamp_utc']
    timestamp_bks = pd.concat([timestamp_bks, df_bkp['timestamp_utc'].tail(1)])

    for idx in df_log.index:
        timestamp_end = df_log.at[idx, 'timestamp_end']
        # if we're not in the same station, skip
        if (timestamp_end < df_bkp.timestamp_utc.min()) or (timestamp_end > df_bkp.timestamp_utc.max()):
            continue

        tds = np.abs(timestamp_bks - timestamp_end)
        iloc_bk = tds.values.argmin()
        if iloc_bk > 0:
            loc_start = timestamp_bks.index[iloc_bk - 1] + 1
        else:
            loc_start = 0

        df_log.at[idx, 'timestamp_ctd_start'] = df_bkp.at[loc_start, 'timestamp_utc']
        df_log.at[idx, 'timestamp_ctd_end'] = timestamp_bks.iloc[iloc_bk]
        df_log.at[idx, 'visit'] = station[-1]
    
    break

In [15]:
df_bkp.drop(columns=['Date', 'Time', 'Site', 'Unit ID', 'User ID'])

,Date,Time,Site,Unit ID,User ID,°C,kPa,DO %,SPC-uS/cm,SAL-PSU,...,NH4-N mg/L,NO3-N mg/L,Cl mg/L,FNU,TSS mg/L,Chl ug/L,PE ug/L,PC ug/L,DEP m,timestamp_utc
0,25/07/11,14:00:42,<None>,NaN,dcs,-0.1,100.54,80.8,7563.0,4.06,...,NaN,NaN,NaN,-0.70,0,NaN,NaN,NaN,0.304,2025-07-11 14:00:42
1,25/07/11,14:00:47,<None>,NaN,dcs,-0.2,100.56,80.6,7334.0,3.93,...,NaN,NaN,NaN,2.10,0,NaN,NaN,NaN,0.317,2025-07-11 14:00:47
2,25/07/11,14:00:52,<None>,NaN,dcs,-0.2,100.56,80.5,7240.0,3.87,...,NaN,NaN,NaN,5.24,0,NaN,NaN,NaN,0.367,2025-07-11 14:00:52
3,25/07/11,14:00:57,<None>,NaN,dcs,-0.3,100.57,80.3,29450.0,17.46,...,NaN,NaN,NaN,7.64,0,NaN,NaN,NaN,1.129,2025-07-11 14:00:57
4,25/07/11,14:01:02,<None>,NaN,dcs,-1.7,100.56,78.4,56467.0,35.11,...,NaN,NaN,NaN,6.40,0,NaN,NaN,NaN,3.308,2025-07-11 14:01:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,25/07/11,16:55:46,<None>,NaN,dcs,-1.7,100.56,93.7,56892.0,35.40,...,NaN,NaN,NaN,-1.53,0,NaN,NaN,NaN,5.345,2025-07-11 16:55:46
1313,25/07/11,16:55:47,<None>,NaN,dcs,-1.7,100.56,93.7,56891.0,35.40,...,NaN,NaN,NaN,-1.54,0,NaN,NaN,NaN,5.285,2025-07-11 16:55:47
1314,25/07/11,16:55:48,<None>,NaN,dcs,-1.7,100.56,93.7,56892.0,35.40,...,NaN,NaN,NaN,-1.53,0,NaN,NaN,NaN,5.248,2025-07-11 16:55:48
1315,25/07/11,16:55:49,<None>,NaN,dcs,-1.7,100.56,93.7,56893.0,35.40,...,NaN,NaN,NaN,-1.53,0,NaN,NaN,NaN,5.240,2025-07-11 16:55:49


In [12]:
df_bkp.columns

Index(['Date', 'Time', 'Site', 'Unit ID', 'User ID', '°C', 'kPa', 'DO %',
       'SPC-uS/cm', 'SAL-PSU', 'pH', 'NH4-N mg/L', 'NO3-N mg/L', 'Cl mg/L',
       'FNU', 'TSS mg/L', 'Chl ug/L', 'PE ug/L', 'PC ug/L', 'DEP m',
       'timestamp'],
      dtype='object')

In [ ]:
df_bkp[['']]

In [11]:
df_log

,date,station,location,position_m,repetition,surface_S_snoworssl_P_pond_RP_refrozenpond,hi_inc_ssl_cm,draft_cm,snoworssl_cm,pond_cm,end_time,notes,timestamp_end,timestamp_ctd_start,timestamp_ctd_end,visit
0,2025-07-11,1,ROV_S,60,1,S,144.0,133.0,tr,NaN,14:04,NaN,2025-07-11 14:04:00,2025-07-11 14:03:31,2025-07-11 14:04:41,a
1,2025-07-11,1,ROV_S,50,1,RP,140.0,133.0,5,NaN,14:16,water drained laterally,2025-07-11 14:16:00,2025-07-11 14:15:56,2025-07-11 14:16:23,a
2,2025-07-11,1,ROV_S,40,1,S,134.0,123.0,tr,NaN,14:29,water drained laterally,2025-07-11 14:29:00,2025-07-11 14:28:53,2025-07-11 14:29:35,a
3,2025-07-11,1,ROV_S,30,1,RP,149.0,139.0,tr,NaN,14:40,NaN,2025-07-11 14:40:00,2025-07-11 14:39:59,2025-07-11 14:40:33,a
4,2025-07-11,1,ROV_S,20,1,S,194.0,179.0,tr,NaN,14:52,"weird depth change, repeated",2025-07-11 14:52:00,2025-07-11 14:51:29,2025-07-11 14:52:24,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2025-08-24,2,transect_rafted,55,1,S,65.0,64.0,NaN,0.0,13:59,no meltwater,2025-08-24 13:59:00,NaT,NaT,
100,2025-08-24,2,t-transect,14,1,S,NaN,72.0,NaN,NaN,14:16,maybe in a pond? Melinda and Nora took this one.,2025-08-24 14:16:00,NaT,NaT,
101,2025-08-26,3,transect,60,1,S,232.0,214.0,4,0.0,15:47,NaN,2025-08-26 15:47:00,NaT,NaT,
102,2025-08-26,3,transect,100,1,RP,162.0,172.0,NaN,13.0,15:26,"Melt pond had 6cm thick lid, cavity was 7cm wi...",2025-08-26 15:26:00,NaT,NaT,


In [10]:
df_log

,date,station,location,position_m,repetition,surface_S_snoworssl_P_pond_RP_refrozenpond,hi_inc_ssl_cm,draft_cm,snoworssl_cm,pond_cm,end_time,notes,timestamp_end,timestamp_ctd_start,timestamp_ctd_end,visit
0,2025-07-11,1,ROV_S,60,1,S,144.0,133.0,tr,NaN,14:04,NaN,2025-07-11 14:04:00,2025-07-11 14:03:31,2025-07-11 14:04:41,a
1,2025-07-11,1,ROV_S,50,1,RP,140.0,133.0,5,NaN,14:16,water drained laterally,2025-07-11 14:16:00,2025-07-11 14:15:56,2025-07-11 14:16:23,a
2,2025-07-11,1,ROV_S,40,1,S,134.0,123.0,tr,NaN,14:29,water drained laterally,2025-07-11 14:29:00,2025-07-11 14:28:53,2025-07-11 14:29:35,a
3,2025-07-11,1,ROV_S,30,1,RP,149.0,139.0,tr,NaN,14:40,NaN,2025-07-11 14:40:00,2025-07-11 14:39:59,2025-07-11 14:40:33,a
4,2025-07-11,1,ROV_S,20,1,S,194.0,179.0,tr,NaN,14:52,"weird depth change, repeated",2025-07-11 14:52:00,2025-07-11 14:51:29,2025-07-11 14:52:24,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2025-08-24,2,transect_rafted,55,1,S,65.0,64.0,NaN,0.0,13:59,no meltwater,2025-08-24 13:59:00,2025-08-24 13:51:18,2025-08-24 13:59:19,d
100,2025-08-24,2,t-transect,14,1,S,NaN,72.0,NaN,NaN,14:16,maybe in a pond? Melinda and Nora took this one.,2025-08-24 14:16:00,2025-08-24 14:10:18,2025-08-24 14:16:20,d
101,2025-08-26,3,transect,60,1,S,232.0,214.0,4,0.0,15:47,NaN,2025-08-26 15:47:00,2025-08-26 15:39:51,2025-08-26 15:47:26,d
102,2025-08-26,3,transect,100,1,RP,162.0,172.0,NaN,13.0,15:26,"Melt pond had 6cm thick lid, cavity was 7cm wi...",2025-08-26 15:26:00,2025-08-26 15:18:59,2025-08-26 15:20:09,d


In [11]:
# Groupby station, visit, location for registry
#df_log.groupby(by=['station', 'visit', 'location'])[['timestamp_ctd_start', 'timestamp_ctd_end']].agg(['min', 'max'])

df_events = pd.concat([df_log.groupby(by=['station', 'visit', 'location'])[['timestamp_ctd_start']].min(),
                       df_log.groupby(by=['station', 'visit', 'location'])[['timestamp_ctd_end']].max(),
                       ], axis=1).reset_index()

df_events['optional_name'] = 'whoi_ysi_' + df_events['location']
df_events['start_formatted'] = df_events['timestamp_ctd_start'].dt.strftime('%Y-%m-%dT%H:%M:%S')
df_events['end_formatted'] = df_events['timestamp_ctd_end'].dt.strftime('%Y-%m-%dT%H:%M:%S')

In [12]:
df_events

,station,visit,location,timestamp_ctd_start,timestamp_ctd_end,optional_name,start_formatted,end_formatted
0,1,a,ROV_M,2025-07-11 15:13:31,2025-07-11 16:55:50,whoi_ysi_ROV_M,2025-07-11T15:13:31,2025-07-11T16:55:50
1,1,a,ROV_S,2025-07-11 14:03:31,2025-07-11 15:03:27,whoi_ysi_ROV_S,2025-07-11T14:03:31,2025-07-11T15:03:27
2,1,b,lead_close,2025-07-28 09:28:26,2025-07-28 09:29:04,whoi_ysi_lead_close,2025-07-28T09:28:26,2025-07-28T09:29:04
3,1,b,lead_far,2025-07-28 08:55:29,2025-07-28 09:01:59,whoi_ysi_lead_far,2025-07-28T08:55:29,2025-07-28T09:01:59
4,1,b,sharedponddark_east,2025-07-27 13:23:51,2025-07-27 13:26:18,whoi_ysi_sharedponddark_east,2025-07-27T13:23:51,2025-07-27T13:26:18
5,1,c,ROV_lead_transect,2025-08-10 14:31:58,2025-08-10 16:32:28,whoi_ysi_ROV_lead_transect,2025-08-10T14:31:58,2025-08-10T16:32:28
6,1,c,coring,2025-08-09 19:30:44,2025-08-09 19:35:14,whoi_ysi_coring,2025-08-09T19:30:44,2025-08-09T19:35:14
7,1,c,lead_transect,2025-08-09 19:54:06,2025-08-09 21:28:09,whoi_ysi_lead_transect,2025-08-09T19:54:06,2025-08-09T21:28:09
8,1,c,past_traps,2025-08-09 18:39:56,2025-08-09 19:24:31,whoi_ysi_past_traps,2025-08-09T18:39:56,2025-08-09T19:24:31
9,2,a,lead0,2025-07-16 14:34:42,2025-07-16 14:39:44,whoi_ysi_lead0,2025-07-16T14:34:42,2025-07-16T14:39:44


In [13]:
df_events.to_csv(os.path.join(data_path, 'registry_events.csv'))